<a href="https://colab.research.google.com/github/sands-eg/GEE_training/blob/main/Dubai_EGS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install geehydro # Life saver for plotting GEE stuff with Python!
!pip install pprint
!pip install geemap

import pprint
import ee
import folium
import geehydro
import geemap
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
import pandas as pd

ERROR: Could not find a version that satisfies the requirement pprint (from versions: none)
ERROR: No matching distribution found for pprint


In [3]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=BR7Fp-HAMmiPoJftYUJ5xUbJmJiinKWKZ_6ib12f0h8&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWjalXT4FQYg-o79fhWjO1whTGCP9aPa2B-T0ZPZWwALUAzNH2k8vwk

Successfully saved authorization token.


In [4]:
# REGION = ee.Geometry.Rectangle(54.8, 24.95, 55.7, 25.5)
# LON_CENTER = 55
# LAT_CENTER = 25

# landsat = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')

# # get collection of landsat images from 2013-2021 for June-August
# landsat = (landsat
#            .filterBounds(REGION)
#            #.filterDate('2013-01-01', '2021-12-31')
#            .filter(ee.Filter.calendarRange(6, 8, "month"))
#            #.filterMetadata('CLOUD_COVER', 'less_than', 50)
#            )
# landsat.size().getInfo()

155

In [42]:
AOI = ee.Geometry.Rectangle(54.8, 24.75, 55.6, 25.4)

# Landsat 8 surface re
dataset = (ee
           .ImageCollection("LANDSAT/LC08/C02/T1_L2")
           .filterBounds(AOI)
           .filter(ee.Filter.lt('CLOUD_COVER', 10))
           .filter(ee.Filter.calendarRange(6, 8, "month"))
          #  .filterDate('2013-01-01', '2013-12-31')
)
print(dataset.size().getInfo())

# visual correction for landsat 8 data
def applyScaleFactors(image): 
  opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermalBand = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  return (image.addBands(opticalBands, None, True)
              .addBands(thermalBand, None, True)
              .addBands(image.metadata('system:time_start')))

dataset = dataset.map(applyScaleFactors)

126


In [46]:
def calc_yearly_composite(year):
  filtered = dataset.filter(ee.Filter.calendarRange(year, year, 'year'))
  # instead of returning the mean over all images one can use quality mosaic to 
  # return the most recent pixel that is unmasked (meaning cloud-free)
  return filtered.qualityMosaic('system:time_start') 

years = ee.List.sequence(2013, 2021)
dataset_ann_L8 = ee.ImageCollection.fromImages(years.map(calc_yearly_composite))
print(dataset_ann_L8.size().getInfo())

9


In [47]:
image = dataset_ann_L8.first().clip(AOI)  
# mosaic = dataset.mosaic().clip(AOI)
# ndvi = image.normalizedDifference(["SR_B4", "SR_B3"])
# viz_params_ndvi = {
#   'min': -1,
#   'max': 1,
#   'palette': ["blue", "white", "green"]
# }
viz_params_rgb = {
  "bands": ['SR_B4', 'SR_B3', 'SR_B2'],
  "min": 0.0,
  "max": 0.3,
}
# Create a default map
Map = geemap.Map()
Map.setCenter(LON_CENTER, LAT_CENTER, 10)
Map.addLayer(image, viz_params_rgb, 'RGB')
# Map.addLayer(mosaic, viz_params_rgb, 'summer mosaic')
#Map.addLayer(ndvi, viz_params_ndvi, 'NDVI')

Map

Map(center=[25, 55], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Tog…

In [48]:
print(dataset_ann_L8.limit(15).getVideoThumbURL({
  'dimensions': 768,
  "region": AOI,
  'crs': 'EPSG:4326',
  "framesPerSecond": 1,
  'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
  'min': 0,
  'max': 0.3
  }))

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/videoThumbnails/868d54773b6e44817191d166038c0a26-d830d042ade424ab4cec90952d416323:getPixels


In [50]:
# Landsat 7 surface reflectance 
dataset = (ee
           .ImageCollection("LANDSAT/LE07/C02/T1_L2")
           .filterBounds(AOI)
           .filter(ee.Filter.lt('CLOUD_COVER', 10))
           .filter(ee.Filter.calendarRange(6, 8, "month"))
          #  .filterDate('2013-01-01', '2013-12-31')
)
print(dataset.size().getInfo())

# visual correction for landsat 7 data
def applyScaleFactors(image): 
  opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermalBand = image.select('ST_B6').multiply(0.00341802).add(149.0)
  return (image.addBands(opticalBands, None, True)
              .addBands(thermalBand, None, True)
              .addBands(image.metadata('system:time_start')))

dataset = dataset.map(applyScaleFactors)

186


In [60]:
years = ee.List.sequence(2000, 2014) # 1999 no data?
dataset_ann_L7 = ee.ImageCollection.fromImages(years.map(calc_yearly_composite))
print(dataset_ann_L7.size().getInfo())

15


In [61]:
image = dataset_ann_L7.first().clip(AOI)  
# mosaic = dataset.mosaic().clip(AOI)
# ndvi = image.normalizedDifference(["SR_B4", "SR_B3"])
# viz_params_ndvi = {
#   'min': -1,
#   'max': 1,
#   'palette': ["blue", "white", "green"]
# }
viz_params_rgb = {
  "bands": ['SR_B3', 'SR_B2', 'SR_B1'],
  "min": 0.0,
  "max": 0.3,
}
# Create a default map
Map = geemap.Map()
Map.setCenter(LON_CENTER, LAT_CENTER, 10)
Map.addLayer(image, viz_params_rgb, 'RGB')
# Map.addLayer(mosaic, viz_params_rgb, 'summer mosaic')
#Map.addLayer(ndvi, viz_params_ndvi, 'NDVI')

Map

Map(center=[25, 55], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Tog…

In [62]:
print(dataset_ann_L7.limit(15).getVideoThumbURL({
  'dimensions': 768,
  "region": AOI,
  'crs': 'EPSG:4326',
  "framesPerSecond": 1,
  'bands': ['SR_B3', 'SR_B2', 'SR_B1'],
  'min': 0,
  'max': 0.3
  }))

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/videoThumbnails/0b21422eb088dba5f736982ad9ee9012-338e5ea74dba7f9867458267d6f582a7:getPixels
